# Using an agent as a callable tool for another agent

Using an agent as a function tool so that one agent can call another as a tool.

You should have the following two environment variables already created and initialized:

* `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI project endpoint
* `AZURE_AI_MODEL_DEPLOYMENT_NAME`: The name of your model deployment

In [ ]:
import asyncio
import os
from random import randint
from agent_framework import ChatAgent, ai_function
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

project_endpoint = os.environ.get('AZURE_AI_PROJECT_ENDPOINT')
model_name = os.environ.get('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print(f"Project endpoint: {project_endpoint}")
print(f"Deployment name: {model_name}")

In [ ]:
from typing import Annotated
from pydantic import Field

@ai_function(name="weather_tool", description="Get the weather for a given location.")
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 72°F."

In [ ]:
# Create the AI Agent Client
client = AzureAIAgentClient(async_credential=AzureCliCredential())

In [ ]:
# Create the Weather agent that can call the tool to get weather

weather_agent = client.create_agent(
    name="WeatherAgent",
    description="An agent that answers questions about the weather.",
    instructions="You answer questions about the weather.",
    tools=get_weather
)

In [ ]:
# Create the main agent that will use the weather agent as a function
main_agent = client.create_agent(
    instructions="You are a helpful assistant who responds in French.",
    name="FrenchHelpfulAssistant",
    tools=weather_agent.as_tool()
)

In [ ]:
result = await main_agent.run("What is the weather like in Amsterdam?")
print(result.text)

In [ ]:
# Cleanup
await client.close()